**单位系统集成 - 解决无量纲参数问题**\n\n你提出的问题很重要：InteractiveIsing.jl的计算是无量纲的，不能直接用真实物理参数。这里展示如何用**Unitful.jl**（已在项目依赖中）来解决这个问题：\n\n**核心思想：建立物理单位→无量纲的转换层**\n\n```julia\nusing Unitful\nusing Unitful: Å, nm, eV, K, T  # 常用单位\n\n# 1. 定义物理参考标尺\nstruct PhysicalScale\n    length_ref::typeof(1.0Å)     # 长度标尺 (如晶格常数)\n    energy_ref::typeof(1.0eV)    # 能量标尺 (如交换能)\n    field_ref::typeof(1.0T)      # 磁场标尺\nend\n\n# PbTiO₃的典型标尺\npbto3_scale = PhysicalScale(3.9Å, 0.1eV, 1.0T)\n\n# 2. 转换函数：物理单位 → 无量纲\ndimensionless(x::Unitful.Length, scale::PhysicalScale) = ustrip(x / scale.length_ref)\ndimensionless(E::Unitful.Energy, scale::PhysicalScale) = ustrip(E / scale.energy_ref)\ndimensionless(B::Unitful.BField, scale::PhysicalScale) = ustrip(B / scale.field_ref)\n\n# 3. 带单位的权重函数定义\nfunction physical_tetragonal_weightfunc(dx, dy, dz; a=3.9Å, c=4.1Å, J₀=0.1eV, scale=pbto3_scale)\n    # 转换为无量纲\n    a_dim = dimensionless(a, scale)\n    c_dim = dimensionless(c, scale)\n    J₀_dim = dimensionless(J₀, scale)\n    \n    if dx == 0 && dy == 0 && dz == 0\n        return 0.0\n    end\n    dr2 = (a_dim*dx)^2 + (a_dim*dy)^2 + (c_dim*dz)^2\n    return J₀_dim / dr2\nend\n\n# 4. 实际使用 - 现在可以直接输入物理参数！\nwg_physical = @WG \"(dx,dy,dz,x,y,z) -> physical_tetragonal_weightfunc(dx,dy,dz; a=3.9Å, c=4.1Å)\" NN=(2,2,2)\n```\n\n**优势：**\n✅ **直接输入物理参数** - 3.9Å, 4.1Å, 0.1eV 等\n✅ **单位安全** - 编译时检查单位匹配\n✅ **标尺灵活** - 不同材料用不同的reference scale\n✅ **向后兼容** - 无量纲版本仍然工作\n\n**进阶应用：**\n- **温度转换**: `k_B * T_physical / E_ref` → 无量纲温度\n- **时间转换**: Monte Carlo步数 ↔ 真实时间\n- **场强转换**: 外加电场 V/m → 无量纲场参数\n\n接下来的代码会同时提供物理版本和无量纲版本！"

**Tutorial 2**

**Before running this code:** Make sure you have the following Julia packages installed: `InteractiveIsing`, `Plots`, `GLMakie`, `FileIO`.

**Install via Julia's package manager**

```julia
using Pkg
Pkg.add(url="https://github.com/rug-minds/InteractiveIsing.jl")
Pkg.add("Plots")
Pkg.add("GLMakie")
Pkg.add("FileIO")
Pkg.add("CairoMakie")
```

**What this tutorial covers:**
This tutorial demonstrates advanced features of InteractiveIsing, including:
- Custom weight functions for different coupling patterns
- Spin glass systems with random and structured disorder

**Prerequisites:**
Please ensure you have completed Tutorial 1 to understand the basic concepts of Ising simulations and hysteresis loops.

In [15]:
## Utility functions for experiments

function newmakie(makietype, args...)
    f = makietype(args...)
    scr = GLMakie.Screen()
    display(scr, f)
    f
end
# Random partition of n into 'parts' segments (returns cumulative sizes)
function random_partition(n, parts)
    if parts == 0
        return [n]
    end
    cuts = sort(rand(1:n-1, parts-1))
    partition = [cuts[1]; diff(cuts); n - cuts[end]]
    return cumsum(partition)
end

# Uniform partition of n into 'parts' nearly equal segments (returns cumulative sizes)
function uniform_partition(n::Integer, parts::Integer)
    if parts == 0
        return [n]
    end
    base = div(n, parts)       # base size of each part
    rem = mod(n, parts)        # remainder to distribute
    partition = [i <= rem ? base + 1 : base for i in 1:parts]
    return cumsum(partition)
end

# Generate random numbers around x with range ±y
function randAround(x::Number, y::Number, dims::Int...)
    x = float(x)
    y = float(y)
    return x .+ y .* (2 .* rand(dims...) .- 1)
end


##################################################################################
### Pulse type: TrianglePulseA (simple four-segment triangular waveform)
struct TrianglePulseA end

function Processes.prepare(::TrianglePulseA, args)
    (;amp, numpulses, rise_point) = args
    steps = num_calls(args)

    first  = LinRange(0, amp, round(Int,rise_point))
    second = LinRange(amp, 0, round(Int,rise_point))
    third  = LinRange(0, -amp, round(Int,rise_point))
    fourth = LinRange(-amp, 0, round(Int,rise_point))
    pulse = vcat(first, second, third, fourth)
    pulse = repeat(pulse, numpulses)

    if steps < length(pulse)
        "Wrong length"
    else
        fix_num = num_calls(args) - length(pulse)
        fix_arr = zeros(Int, fix_num)
        pulse   = vcat(pulse, fix_arr)
    end

    # Predefine storage arrays
    x = Float32[]
    y = Float32[]
    processsizehint!(args, x)
    processsizehint!(args, y)

    return (;pulse, x, y)
end

function (::TrianglePulseA)(args)
    (;pulse, M, x, y, hamiltonian) = args
    pulse_val = pulse[algo_loopidx(args)]
    hamiltonian.b[] = pulse_val
    push!(x, pulse_val)
    push!(y, M[])
end


##################################################################################
### Pulse type: PUNDa (no delay time between pulses)
struct PUNDa end

function Processes.prepare(::PUNDa, args)
    (;amp, numpulses, rise_point) = args
    steps = num_calls(args)

    first  = LinRange(0, amp, round(Int,rise_point))
    second = LinRange(amp, 0, round(Int,rise_point))
    third  = LinRange(0, -amp, round(Int,rise_point))
    fourth = LinRange(-amp, 0, round(Int,rise_point))
    pulse = vcat(first, second, first, second, third, fourth, third, fourth)
    pulse = repeat(pulse, numpulses)

    if steps < length(pulse)
        println("Pulse is longer than lifetime")
    else
        fix_num = num_calls(args) - length(pulse)
        fix_arr = zeros(Int, fix_num)
        pulse   = vcat(pulse, fix_arr)
    end

    # Predefine storage arrays
    x = Float32[]
    y = Float32[]
    all_Es = Float32[]
    processsizehint!(args, x)
    processsizehint!(args, y)

    return (;pulse, x, y)
end

function (::PUNDa)(args)
    (;pulse, M, x, y, hamiltonian) = args
    pulse_val = pulse[algo_loopidx(args)]
    hamiltonian.b[] = pulse_val
    push!(x, pulse_val)
    push!(y, M[])
end


##################################################################################
### Pulse type: PUNDb (with delay time between each pulse)
struct PUNDb end

function Processes.prepare(::PUNDb, args)
    (;amp, numpulses, rise_point) = args
    steps = num_calls(args)

    delay  = zeros(Int, rise_point)
    first  = LinRange(0, amp, round(Int,rise_point))
    amp1   = fill(amp, rise_point)
    second = LinRange(amp, 0, round(Int,rise_point))
    third  = LinRange(0, -amp, round(Int,rise_point))
    amp2   = fill(-amp, rise_point)
    fourth = LinRange(-amp, 0, round(Int,rise_point))

    pulse = vcat(delay, first, amp1, second,
                 delay, first, amp1, second,
                 delay, third, amp2, fourth,
                 delay, third, amp2, fourth)

    pulse = repeat(pulse, numpulses)

    if steps < length(pulse)
        println("Pulse is longer than lifetime")
    else
        fix_num = num_calls(args) - length(pulse)
        fix_arr = zeros(Int, fix_num)
        pulse   = vcat(pulse, fix_arr)
    end

    # Predefine storage arrays
    x = Float32[]
    y = Float32[]
    all_Es = Float32[]
    processsizehint!(args, x)
    processsizehint!(args, y)

    return (;pulse, x, y)
end

function (::PUNDb)(args)
    (;pulse, M, x, y, hamiltonian) = args
    pulse_val = pulse[algo_loopidx(args)]
    hamiltonian.b[] = pulse_val
    push!(x, pulse_val)
    push!(y, M[])
end


In [16]:
# Setup: Load packages and create basic 3D Ising graph
# This follows the same pattern as Tutorial 1

using InteractiveIsing, Plots, GLMakie, FileIO
using InteractiveIsing.Processes
import InteractiveIsing as II

# Initialize a 3D Ising graph with specified dimensions and periodic boundary conditions
xL = 100  # Length in the x-dimension
yL = 100  # Length in the y-dimension
zL = 10   # Length in the z-dimension (thin film geometry)

10

In [ ]:
# ========================================================================\n# 物理单位集成 - Unitful.jl 实现\n# ========================================================================\n\nusing Unitful\nusing Unitful: Å, nm, eV, K, T, V, m  # 导入常用单位\n\n# 1. 定义物理参考标尺结构\nstruct PhysicalScale\n    length_ref::typeof(1.0Å)     # 长度标尺 (通常选主要晶格常数)\n    energy_ref::typeof(1.0eV)    # 能量标尺 (交换耦合强度)\n    field_ref::typeof(1.0V/m)    # 电场标尺\n    temp_ref::typeof(1.0K)       # 温度标尺\nend\n\n# 常见材料的标尺\npbto3_scale = PhysicalScale(3.9Å, 0.1eV, 1e6V/m, 300K)  # PbTiO₃\nbatio3_scale = PhysicalScale(4.0Å, 0.05eV, 5e5V/m, 400K)  # BaTiO₃\ngeneric_scale = PhysicalScale(1.0Å, 1.0eV, 1.0V/m, 1.0K)   # 通用标尺\n\n# 2. 转换函数：物理单位 → 无量纲\ndimensionless(x::Unitful.Length, scale::PhysicalScale) = ustrip(x / scale.length_ref)\ndimensionless(E::Unitful.Energy, scale::PhysicalScale) = ustrip(E / scale.energy_ref)\ndimensionless(F::Unitful.EField, scale::PhysicalScale) = ustrip(F / scale.field_ref)\ndimensionless(T::Unitful.Temperature, scale::PhysicalScale) = ustrip(T / scale.temp_ref)\n\n# 3. 逆转换：无量纲 → 物理单位\nwith_units(x_dim::Number, ::typeof(1.0Å), scale::PhysicalScale) = x_dim * scale.length_ref\nwith_units(E_dim::Number, ::typeof(1.0eV), scale::PhysicalScale) = E_dim * scale.energy_ref\n\n# 4. 带单位的权重函数 - 正方晶系\nfunction physical_tetragonal_weightfunc(dx, dy, dz; \n                                       a=3.9Å, c=4.1Å, J₀=0.1eV, \n                                       scale=pbto3_scale)\n    \"\"\"物理参数版本的正方晶系权重函数\"\"\"\n    # 转换为无量纲\n    a_dim = dimensionless(a, scale)\n    c_dim = dimensionless(c, scale)\n    J₀_dim = dimensionless(J₀, scale)\n    \n    if dx == 0 && dy == 0 && dz == 0\n        return 0.0\n    end\n    dr2 = (a_dim*dx)^2 + (a_dim*dy)^2 + (c_dim*dz)^2\n    return J₀_dim / dr2\nend\n\n# 5. 温度转换辅助函数\nfunction physical_temp_to_dimensionless(T_phys::Unitful.Temperature, scale::PhysicalScale)\n    \"\"\"将物理温度转换为Monte Carlo中使用的无量纲温度\n    \n    注意：这里假设 k_B * T / J₀ 作为无量纲温度\n    实际转换因子可能需要根据具体哈密顿量调整\n    \"\"\"\n    k_B = 8.617e-5eV/K  # 玻尔兹曼常数\n    return ustrip((k_B * T_phys) / scale.energy_ref)\nend\n\n# 6. 示例：从实验数据创建权重生成器\nfunction create_physical_wg(material_name::String)\n    if material_name == \"PbTiO3\"\n        # 实验数据：PbTiO₃ 室温\n        a, c = 3.898Å, 4.135Å\n        J₀ = 0.12eV\n        scale = pbto3_scale\n    elseif material_name == \"BaTiO3\"\n        # 实验数据：BaTiO₃ 室温 \n        a, c = 3.99Å, 4.03Å\n        J₀ = 0.08eV\n        scale = batio3_scale\n    else\n        error(\"Unknown material: $material_name\")\n    end\n    \n    return @WG \"(dx,dy,dz,x,y,z) -> physical_tetragonal_weightfunc(dx,dy,dz; a=$(a), c=$(c), J₀=$(J₀), scale=$(scale))\" NN=(2,2,2)\nend\n\nprintln(\"物理单位系统已加载！\")\nprintln(\"用法示例：\")\nprintln(\"  wg_pbti = create_physical_wg(\\\"PbTiO3\\\")\")\nprintln(\"  T_dimensionless = physical_temp_to_dimensionless(300K, pbto3_scale)\")\n\n# 创建几个常用的物理权重生成器\nwg_pbti_physical = create_physical_wg(\"PbTiO3\")\nwg_bati_physical = create_physical_wg(\"BaTiO3\")\n\n# 展示转换结果\nT_room = 300K\nT_dim_pbti = physical_temp_to_dimensionless(T_room, pbto3_scale)\nT_dim_bati = physical_temp_to_dimensionless(T_room, batio3_scale)\n\nprintln(\"\\n转换示例：\")\nprintln(\"室温 $(T_room) 在 PbTiO₃ 标尺下 = $(round(T_dim_pbti, digits=3)) (无量纲)\")\nprintln(\"室温 $(T_room) 在 BaTiO₃ 标尺下 = $(round(T_dim_bati, digits=3)) (无量纲)\")"

In [ ]:
# ========================================================================\n# 快速参数配置 - 解决运行时间问题\n# ========================================================================\n\n# 参数预设：从快速演示到高精度研究\nstruct SimParams\n    xL::Int\n    yL::Int \n    zL::Int\n    risepoint::Int\n    PulseN::Int\n    Time_fctr::Float64\n    Amplitude::Float64\n    c_depol::Float64\n    temp::Float64\nend\n\n# 预设配置\nconst QUICK_DEMO = SimParams(\n    30, 30, 3,     # 小网格 (2.7k 自旋)\n    50,            # 快速脉冲上升\n    1,             # 单个脉冲\n    0.01,          # 低时间因子\n    10.0,          # 中等振幅\n    30000.0,       # 中等去极化\n    1.2            # 较高温度利于快速平衡\n)\n\nconst STANDARD = SimParams(\n    50, 50, 5,     # 中等网格 (12.5k 自旋)\n    200,           # 标准脉冲\n    2,             # 双脉冲\n    0.05,          # 标准时间因子\n    15.0,          # 标准振幅\n    60000.0,       # 标准去极化\n    1.5            # 标准温度\n)\n\nconst HIGH_QUALITY = SimParams(\n    100, 100, 10,  # 大网格 (100k 自旋)\n    500,           # 精细脉冲\n    3,             # 多脉冲\n    0.2,           # 高时间因子\n    20.0,          # 高振幅\n    100000.0,      # 强去极化\n    1.0            # 低温高精度\n)\n\n# 运行时间估算函数\nfunction estimate_runtime(params::SimParams)\n    n_spins = params.xL * params.yL * params.zL\n    speed_rate = Int(params.Time_fctr * n_spins)\n    total_steps = (params.PulseN * 4 + 2) * params.risepoint * speed_rate\n    \n    # 粗略估算：每秒处理约10^5步\n    estimated_seconds = total_steps / 1e5\n    \n    if estimated_seconds < 60\n        return \"$(round(estimated_seconds, digits=1)) 秒\"\n    elseif estimated_seconds < 3600\n        return \"$(round(estimated_seconds/60, digits=1)) 分钟\"\n    else\n        return \"$(round(estimated_seconds/3600, digits=1)) 小时\"\n    end\nend\n\n# 创建图形和设置参数的便捷函数\nfunction setup_simulation(params::SimParams; periodic=(:x,:y), \n                         self_energy=-1.0, marker_size=0.3)\n    \n    println(\"正在设置仿真...\")\n    println(\"网格: $(params.xL)×$(params.yL)×$(params.zL) = $(params.xL*params.yL*params.zL) 自旋\")\n    println(\"估计运行时间: $(estimate_runtime(params))\")\n    \n    # 创建图\n    g = IsingGraph(params.xL, params.yL, params.zL, type = Continuous(), periodic = periodic)\n    \n    # 设置可视化\n    II.makie_markersize[] = marker_size\n    interface(g)\n    \n    # 设置哈密顿量\n    g.hamiltonian = Ising(g) + DepolField(g, c=params.c_depol, left_layers=1, right_layers=1)\n    g.hamiltonian = sethomogenousparam(g.hamiltonian, :b)\n    \n    # 设置参数\n    homogeneousself!(g, self_energy)\n    settemp(g, params.temp)\n    \n    return g\nend\n\n# 统一的仿真运行函数\nfunction run_hysteresis(g, wg, params::SimParams; show_progress=true)\n    println(\"\\n开始仿真: $(estimate_runtime(params))\")\n    \n    # 应用权重生成器\n    genAdj!(g[1], wg)\n    \n    # 计算仿真参数\n    fullsweep = params.xL * params.yL * params.zL\n    SpeedRate = Int(params.Time_fctr * fullsweep)\n    Pulsetime = (params.PulseN * 4 + 2) * params.risepoint * SpeedRate\n    \n    # 创建并运行流程\n    compalgo = CompositeAlgorithm((Metropolis, TrianglePulseA), (1, SpeedRate))\n    createProcess(g, compalgo, \n                 lifetime = Pulsetime, \n                 amp = params.Amplitude, \n                 numpulses = params.PulseN, \n                 rise_point = params.risepoint)\n    \n    if show_progress\n        est_remaining(process(g))\n    end\n    \n    # 等待完成\n    args = process(g) |> fetch\n    \n    return args.x, args.y  # voltage, Pr\nend\n\n# 快速对比函数\nfunction compare_weight_functions(wg_list, labels, params=QUICK_DEMO; \n                                 title=\"Weight Function Comparison\")\n    results = []\n    \n    for (i, (wg, label)) in enumerate(zip(wg_list, labels))\n        println(\"\\n=== 运行 $label ===\")\n        g = setup_simulation(params)\n        \n        voltage, Pr = run_hysteresis(g, wg, params)\n        push!(results, (voltage, Pr, label))\n        \n        println(\"完成 $label\")\n    end\n    \n    # 绘制对比图\n    fig = Figure(resolution=(800, 600))\n    ax = Axis(fig[1,1], xlabel=\"Voltage\", ylabel=\"Polarization\", title=title)\n    \n    colors = [:red, :blue, :green, :orange, :purple]\n    for (i, (voltage, Pr, label)) in enumerate(results)\n        lines!(ax, voltage, Pr, label=label, color=colors[i], linewidth=2)\n    end\n    \n    axislegend(ax)\n    display(fig)\n    \n    return results\nend\n\nprintln(\"快速参数系统已加载！\")\nprintln(\"\\n可用配置：\")\nprintln(\"  QUICK_DEMO:  $(estimate_runtime(QUICK_DEMO)) - 快速演示\")\nprintln(\"  STANDARD:    $(estimate_runtime(STANDARD)) - 标准质量\")\nprintln(\"  HIGH_QUALITY: $(estimate_runtime(HIGH_QUALITY)) - 高精度研究\")\nprintln(\"\\n使用示例：\")\nprintln(\"  g = setup_simulation(QUICK_DEMO)\")\nprintln(\"  voltage, Pr = run_hysteresis(g, wg_glass2, QUICK_DEMO)\")\nprintln(\"  compare_weight_functions([wg_glass1, wg_glass2], [\\\"Ferro\\\", \\\"Anti\\\"], QUICK_DEMO)\")"

**Custom Weight Functions**

Weight functions determine the coupling strength between neighboring spins in the Ising model. They are the core mechanism for defining different interaction patterns and building the adjacency matrix.

**How Weight Functions Work in the Code:**

The `@WG` macro creates a `WeightGenerator` that calls your weight function for each pair of dipoles. Here's what actually happens:

1. **For each dipole at position `(x, y, z)`**:
   - Find all neighbors within `NN` range: `(x+dx, y+dy, z+dz)`
   - Call your weight function with parameters: `(dx, dy, dz, x, y, z, ...)`
   - Store the **returned coupling strength** in the adjacency matrix

2. **The weight function returns the actual coupling coefficient `J_ij`**:
   - This value goes directly into the Hamiltonian: `H = -∑ J_ij s_i s_j`
   - Positive values → ferromagnetic coupling (parallel spins favored)
   - Negative values → antiferromagnetic coupling (anti-parallel spins favored)

**Understanding the Distance Scaling `dr2`:**

```julia
dr2 = (rx*dx)^2 + (ry*dy)^2 + (rz*dz)^2
return prefactor / dr2  # This J_ij goes into the Hamiltonian
```

**What `dr2` represents:**
- **`(dx, dy, dz)`**: Relative displacement in lattice units (e.g., dx=1 means "one lattice step right")
- **`(rx, ry, rz)`**: **User-defined scaling factors** - you control these!
- **`dr2`**: Your custom distance metric for determining coupling strength

**Key Insight - You Define the Scaling Rule:**

The scaling factors `(rx, ry, rz)` are **entirely up to you**. They let you:
- **Approximate realistic dipole interactions**: Use physical lattice constants  
- **Create anisotropic coupling**: Different strengths along x, y, z directions
- **Model material properties**: Account for crystal structure, defects, etc.
- **Design artificial patterns**: Create specific domain structures

**Examples of Distance Scaling:**
- **Isotropic**: `rx = ry = rz = 1` → uniform spacing in all directions
- **Anisotropic**: `rx = 1, ry = 1, rz = 0.5` → compressed interactions along z-axis  
- **Physical units**: `rx = 3.9Å, ry = 3.9Å, rz = 4.1Å` → real lattice constants
- **Effective scaling**: `rx = 1.2, ry = 0.8, rz = 1.0` → tuned for specific material behavior

**Why Use 1/r² Decay:**

This is a **modeling choice** to approximate how dipole interaction energy decays with distance:
- **Closer dipoles** → smaller `dr2` → larger coupling `|J_ij|`
- **Farther dipoles** → larger `dr2` → weaker coupling `|J_ij|`
- **Balance**: Strong enough to capture nearest-neighbor physics, weak enough for computational stability

**The Power is in Your Hands:**
You can use any distance function you want: `1/dr2`, `1/dr2^1.5`, `exp(-dr2)`, or even completely custom rules. The weight function returns the final coupling strength that determines how strongly each pair of dipoles interacts.

In [ ]:
# Weight function variant 1: Pure ferromagnetic coupling
function weightfunc1(dx, dy, dz, rx, ry, rz)
    # Always positive coupling (ferromagnetic)
    dr2 = (rx*dx)^2 + (ry*dy)^2 + (rz*dz)^2
    return 1.0 / dr2
end

# Weight function variant 2: Checkerboard antiferromagnetic pattern
function weightfunc2(dx, dy, dz, rx, ry, rz)
    # Alternating signs based on lattice position parity
    dr2 = (rx*dx)^2 + (ry*dy)^2 + (rz*dz)^2
    
    # Checkerboard pattern: alternating ferro/antiferro based on coordinate sum
    if (abs(dx) + abs(dy)) % 2 == 0
        return 1.0 / dr2    # Ferromagnetic coupling
    else
        return -1.0 / dr2   # Antiferromagnetic coupling
    end
end

weightfunc2 (generic function with 1 method)

**Advanced Weight Functions: Spin Glass Systems**

Now we explore complex interaction patterns that create spin glass behavior and domain structures by using **spatially-varying coupling parameters**.

**Understanding Weight Function Parameters:**

Weight functions can take many parameters. The most common pattern is:
```julia
function my_weight_func(dx, dy, dz, x, y, z, additional_params...)
```

**Parameter Roles:**
- **`(dx, dy, dz)`**: **Relative displacement** to the neighbor (e.g., dx=1 means "one step right")
- **`(x, y, z)`**: **Absolute coordinates** of the central dipole in the lattice
- **`additional_params...`**: **Your custom parameters** (domain boundaries, random coefficients, etc.)

**Creating Spatial Disorder:**

The key insight is that you can make the coupling strength **location-dependent**:

1. **Divide the lattice into domains** using partition functions:
   ```julia
   x_n = random_partition(xL, 20)  # 20 random domains along x
   y_n = uniform_partition(yL, 10) # 10 uniform domains along y
   ```

2. **Assign domain-specific properties**:
   ```julia
   coef_x = randAround(1, 0.2, 20)  # Random scaling around 1.0 ± 0.2
   coef_y = randAround(1, 0.2, 10)  # Different for each domain
   ```

3. **Use absolute coordinates `(x, y, z)` to determine which domain**:
   ```julia
   ix = findfirst(v -> x ≤ v, x_n)  # Which x-domain contains this dipole?
   iy = findfirst(v -> y ≤ v, y_n)  # Which y-domain contains this dipole?
   ```

4. **Apply domain-specific scaling to the distance**:
   ```julia
   dr2 = (coef_x[ix] * dx)^2 + (coef_y[iy] * dy)^2  # Domain-dependent distance
   ```

**Physical Interpretation:**

This creates **realistic material disorder**:
- **Different domains** have different "effective lattice constants" 
- **Strain/stress variations**: Some regions compressed (`coef < 1`), others stretched (`coef > 1`)
- **Chemical inhomogeneity**: Different local compositions change interaction strength
- **Defects and interfaces**: Domain boundaries create frustration and pinning sites

**The Result: Spin Glass Behavior**

When you combine:
- **Spatial disorder** in coupling strengths (different `coef` values)
- **Competing interactions** (ferromagnetic vs antiferromagnetic patterns)  
- **Long-range connections** (beyond nearest neighbors)

You get **frustrated energy landscapes** where:
- No single configuration can satisfy all interactions
- Multiple local energy minima exist
- System shows complex, history-dependent switching behavior

**Practical Example: Domain-Based Coupling**

Let's see how `weightfunc_glass_anti()` actually works:

```julia
function weightfunc_glass_anti(dx, dy, dz, x, y, z, x_n, y_n, z_n, coef_x, coef_y, coef_z)
    # Step 1: Determine which domain this dipole belongs to
    ix = findfirst(v -> x ≤ v, x_n)  # Domain index along x
    iy = findfirst(v -> y ≤ v, y_n)  # Domain index along y  
    iz = findfirst(v -> z ≤ v, z_n)  # Domain index along z
    
    # Step 2: Use domain-specific scaling coefficients
    dr2 = (coef_x[ix] * dx)^2 + (coef_y[iy] * dy)^2 + (coef_z[iz] * dz)^2
    
    # Step 3: Apply interaction pattern (checkerboard antiferromagnetic)
    prefac = (abs(dx) + abs(dy)) % 2 != 0 ? -1 : 1
    
    # Step 4: Return the coupling strength J_ij
    return prefac / dr2
end
```

**What This Achieves:**

1. **Heterogeneous Coupling Strengths**: 
   - Domain A: `coef_x = 1.2` → weaker interactions (`J ∝ 1/1.44`)
   - Domain B: `coef_x = 0.8` → stronger interactions (`J ∝ 1/0.64`)

2. **Realistic Material Properties**:
   - Models regions with different local environments
   - Creates natural disorder without completely random coupling
   - Maintains physical distance-dependence within each domain

3. **Controlled Frustration**:
   - Systematic rather than purely random disorder
   - Allows investigation of specific domain effects
   - Can model real material interfaces and defects

**Key Advantage**: You're not just creating random couplings - you're building **physically motivated spatial patterns** that capture realistic material heterogeneity while maintaining the essential physics of distance-dependent interactions.

**Advanced Application: Crystal Structure Modeling**

Beyond spin glass systems, weight functions can model **real ferroelectric crystal structures** by incorporating actual crystallographic parameters. This bridges atomistic structure with mesoscopic behavior.

**Crystal Structure Examples:**

Different crystal systems have distinct lattice parameters and symmetries that directly affect ferroelectric properties:

1. **Cubic Crystals** (e.g., BaTiO₃ high-temperature phase):
   - Equal lattice constants: a = b = c
   - Isotropic interactions in all directions
   - No spontaneous polarization (paraelectric)

2. **Tetragonal Crystals** (e.g., PbTiO₃, BaTiO₃ room temperature):
   - a = b ≠ c (elongated or compressed along one axis)
   - Enhanced coupling along the unique axis
   - Spontaneous polarization along c-axis

3. **Orthorhombic Crystals** (e.g., distorted perovskites):
   - a ≠ b ≠ c (three different lattice constants)
   - Complex anisotropic behavior
   - Multiple possible polarization directions

4. **Layered Structures** (e.g., Aurivillius phases):
   - Strong in-plane coupling
   - Weak interlayer interactions
   - Quasi-2D ferroelectric behavior

**Key Advantages:**
- **Quantitative modeling** using experimental lattice constants
- **Temperature-dependent** structural transitions
- **Structure-property** relationship investigations
- **Material comparison** and design

In [ ]:
# ========================================================================
# CRYSTAL STRUCTURE WEIGHT FUNCTIONS
# ========================================================================

# Cubic crystal structure (e.g., BaTiO₃ cubic phase)
function cubic_weightfunc(dx, dy, dz, a = 4.0)
    """
    Cubic crystal: a = b = c, α = β = γ = 90°
    Example: BaTiO₃ cubic phase (a ≈ 4.0 Å)
    """
    dr2 = (a*dx)^2 + (a*dy)^2 + (a*dz)^2
    return 1.0 / dr2
end

# Tetragonal crystal structure (e.g., PbTiO₃)
function tetragonal_weightfunc(dx, dy, dz, a = 3.9, c = 4.1)
    """
    Tetragonal crystal: a = b ≠ c, α = β = γ = 90°
    Example: PbTiO₃ (a ≈ 3.9 Å, c ≈ 4.1 Å)
    c/a ≈ 1.05 indicates elongation along z-axis
    """
    dr2 = (a*dx)^2 + (a*dy)^2 + (c*dz)^2
    return 1.0 / dr2
end

# Orthorhombic crystal structure (e.g., GdFeO₃-type)
function orthorhombic_weightfunc(dx, dy, dz, a = 3.8, b = 3.9, c = 4.0)
    """
    Orthorhombic crystal: a ≠ b ≠ c, α = β = γ = 90°
    Example: Distorted perovskite structures
    """
    if dx == 0 && dy == 0 && dz == 0
        return 0.0
    end
    dr2 = (a*dx)^2 + (b*dy)^2 + (c*dz)^2
    return 1.0 / dr2
end

# Layered structure with reduced interlayer coupling
function layered_weightfunc(dx, dy, dz, a = 3.9, c = 25.0, λ = 0.1)
    """
    Layered structure with weak interlayer coupling
    Example: Aurivillius phases, Dion-Jacobson phases
    λ = interlayer coupling reduction factor
    """
    dr2 = (a*dx)^2 + (a*dy)^2 + (c*dz)^2
    
    # Reduce coupling for interlayer interactions (dz ≠ 0)
    interlayer_factor = (dz == 0) ? 1.0 : λ
    return interlayer_factor / dr2
end

# Example weight generators for different crystal structures
println("Creating weight generators for crystal structures...")

# Cubic BaTiO₃-like
wg_cubic = @WG "(dx,dy,dz,x,y,z) -> cubic_weightfunc(dx,dy,dz,4.0)" NN = (2,2,2)

# Tetragonal PbTiO₃-like  
wg_tetragonal = @WG "(dx,dy,dz,x,y,z) -> tetragonal_weightfunc(dx,dy,dz,3.9,4.1)" NN = (2,2,2)

# Orthorhombic distorted perovskite
wg_orthorhombic = @WG "(dx,dy,dz,x,y,z) -> orthorhombic_weightfunc(dx,dy,dz,3.8,3.9,4.0)" NN = (2,2,2)

# Layered structure
wg_layered = @WG "(dx,dy,dz,x,y,z) -> layered_weightfunc(dx,dy,dz,3.9,25.0,0.1)" NN = (2,2,2)

println("Crystal structure weight generators created!")
println("Usage: genAdj!(g[1], wg_tetragonal)  # Use any of: wg_cubic, wg_tetragonal, wg_orthorhombic, wg_layered")

**Research Applications with Crystal Structure Models:**

 

These crystal structure weight functions enable **quantitative ferroelectric research**:

 

🔬 **Experimental Validation**: 

- Use X-ray diffraction data → lattice parameters → simulation input

- Compare simulated vs experimental domain patterns

 

📊 **Structure-Property Studies**:

- How does c/a ratio affect coercive field in tetragonal crystals?

- What switching mechanisms occur in orthorhombic systems?

- How do layered structures influence polarization fatigue?

 

🌡️ **Phase Transition Modeling**:

- Cubic ↔ Tetragonal transitions (temperature-dependent parameters)

- Structural evolution during field cycling

- Thermal expansion effects

 

💡 **Material Design**:

- Optimize lattice parameters for specific applications

- Predict properties of new compositions

- Guide synthesis of engineered structures

 

---

 

⚠️ **Important note: limitations of the dr2 scaling model**

 

A simple 1/dr² distance decay is **not** universally appropriate.

 

**Physical considerations:**

1. Real dipole–dipole interaction scales as ~1/r³; using 1/r² is an approximation

2. Exchange interactions often decay exponentially ~e^(−r/a) (strongly correlated systems)

3. Electrostatic screening can be significant in high‑κ materials

4. Quantum tunneling may matter in thin films or nanostructures

 

**When to modify the distance law:**

- Strongly correlated oxides: consider exponential decay, e.g., exp(−√dr2/λ)

- Near metallic gates/electrodes: include image‑charge corrections

- Surfaces and interfaces: broken symmetry induces anisotropy

- Quantum dot arrays: tunneling‑dominated coupling

 

**Suggested alternative distance laws (sketches):**

```julia

# Exponential decay (strongly correlated systems)

return J₀ * exp(-sqrt(dr2)/λ)

 

# Modified dipolar interaction

return J₀ / (dr2^1.5)  # closer to 1/r³

 

# Screened Coulomb interaction

return J₀ * exp(-sqrt(dr2)/λ_screen) / dr2

 

# Anisotropy for layered materials

aniso_factor = (dz == 0) ? 1.0 : exp(-abs(dz)/d_layer)

return J₀ * aniso_factor / dr2

```

 

**Physical sanity checklist when defining weight functions:**

✅ Does the distance decay follow known physics?  

✅ Is the anisotropy consistent with crystal symmetry?  

✅ Are boundary conditions reasonable?  

✅ Numerically stable (avoid divergences)?

 

---

 

**🚀 Suggestions: what else to explore?**

 

1. **Multiscale modeling** 🔬

- Atomistic → mesoscopic: from DFT to continuum models

- Time scales: femtosecond pulses → quasi‑static switching

- Spatial scales: nanoscale domain walls → device scale (mm)

 

2. **Machine‑learning augmentation** 🤖

- Automatic parameter tuning: fit weight functions to experiments

- Phase‑transition prediction: infer Tc from structural parameters

- Inverse design: given performance targets, design optimal structures

 

3. **Non‑equilibrium dynamics** ⚡

- Ultrafast switching: domain evolution under femtosecond pulses

- Fatigue mechanisms: micro‑damage accumulation under cycling

- Thermal effects: Joule heating impact on switching

 

4. **Topological ferroelectrics** 🌀

- Domain‑wall topology: vortices, skyrmions

- Topological transitions: continuous deformation of domain textures

- Protected states: topologically robust ferroelectric structures

 

5. **Heterostructure engineering** 🔧

- Interfacial coupling: ferroelectric/magnetic/superconducting multilayers

- Strain engineering: substrate‑induced phases

- Size effects: ferroelectricity under quantum confinement

 

6. **Experiment–theory loop** 🔄

- Real‑time feedback: auto‑update model parameters from data

- Predict→verify cycle: theory → experiment → model refinement

- Digital twins: virtual replicas of real devices

 

**Most promising directions (my take):**

🎯 Multiphysics coupling: electro‑thermal‑mechanical‑optical co‑modeling  

🎯 Materials informatics: high‑throughput workflows + data mining  

🎯 Quantum ferroelectrics: phenomena dominated by quantum effects

 

**Practical next steps:**

1. Choose a specific material (e.g., PbTiO₃)

2. Collect experimental inputs (lattice constants, domain patterns)

3. Implement a custom weight function

4. Compare simulation vs experiment

5. Look for new physical insights!

 

**Next Steps**: Try replacing `wg_glass2` below with `wg_tetragonal` to see how crystal structure affects hysteresis behavior!


In [ ]:
# ========================================================================
# More Physically Realistic Distance Functions Examples
# ========================================================================

# Exponential decay (strongly correlated systems, e.g., transition metal oxides)
function exponential_decay_weightfunc(dx, dy, dz, a = 3.9, λ = 2.0, J₀ = 1.0)
    """
    Exponential decay interaction: J ∝ exp(-r/λ)
    Applicable to: strongly correlated electron systems, exchange interactions
    λ: correlation length, typically a few lattice constants
    """
    if dx == 0 && dy == 0 && dz == 0
        return 0.0
    end
    r = sqrt((a*dx)^2 + (a*dy)^2 + (a*dz)^2)
    return J₀ * exp(-r/λ)
end

# Real dipole interaction (1/r³ decay)
function dipole_interaction_weightfunc(dx, dy, dz, a = 3.9, J₀ = 1.0)
    """
    Real dipole-dipole interaction: J ∝ 1/r³
    Closer to real long-range electrostatic interactions
    Note: numerically need to carefully handle nearest-neighbor divergence
    """
    if dx == 0 && dy == 0 && dz == 0
        return 0.0
    end
    dr2 = (a*dx)^2 + (a*dy)^2 + (a*dz)^2
    r = sqrt(dr2)
    # Add small cutoff to avoid numerical divergence
    r_cutoff = max(r, 0.5*a)
    return J₀ / (r_cutoff^3)
end

# Screened Coulomb interaction (e.g., Thomas-Fermi screening)
function screened_coulomb_weightfunc(dx, dy, dz, a = 3.9, λ_screen = 5.0, J₀ = 1.0)
    """
    Screened Coulomb interaction: J ∝ exp(-r/λ)/r
    Applicable to: metallic systems, high carrier density materials
    λ_screen: screening length (Debye length)
    """
    if dx == 0 && dy == 0 && dz == 0
        return 0.0
    end
    r = sqrt((a*dx)^2 + (a*dy)^2 + (a*dz)^2)
    return J₀ * exp(-r/λ_screen) / r
end

# Anisotropic layered structure
function anisotropic_layered_weightfunc(dx, dy, dz, a = 3.9, c = 25.0, d_layer = 2.0, J₀ = 1.0)
    """
    Anisotropic layered structure
    Intralayer: normal interactions
    Interlayer: exponential decay
    d_layer: interlayer coupling decay length
    """
    if dx == 0 && dy == 0 && dz == 0
        return 0.0
    end
    
    # Intralayer interactions (dz = 0)
    if dz == 0
        dr2 = (a*dx)^2 + (a*dy)^2
        return J₀ / (dr2 + (0.5*a)^2)  # Add small regularization
    else
        # Interlayer interactions (dz ≠ 0)
        dr2 = (a*dx)^2 + (a*dy)^2 + (c*dz)^2
        interlayer_decay = exp(-abs(c*dz)/d_layer)
        return J₀ * interlayer_decay / (dr2 + (0.5*a)^2)
    end
end

# Strain-modulated interactions
function strain_modulated_weightfunc(dx, dy, dz, x, y, z, a = 3.9, strain_grad = 0.01, J₀ = 1.0)
    """
    Strain gradient modulated interactions
    Simulate substrate strain, thermal stress effects
    strain_grad: strain gradient strength
    """
    if dx == 0 && dy == 0 && dz == 0
        return 0.0
    end
    
    # Local strain varies with position
    local_strain = 1.0 + strain_grad * (x/100.0)  # normalized position
    effective_a = a * local_strain
    
    dr2 = (effective_a*dx)^2 + (effective_a*dy)^2 + (effective_a*dz)^2
    return J₀ / dr2
end

println("Creating more physically realistic weight functions...")

# Create these advanced weight generators
wg_exponential = @WG "(dx,dy,dz,x,y,z) -> exponential_decay_weightfunc(dx,dy,dz,3.9,2.0,1.0)" NN = (3,3,3)
wg_dipole = @WG "(dx,dy,dz,x,y,z) -> dipole_interaction_weightfunc(dx,dy,dz,3.9,1.0)" NN = (3,3,3)
wg_screened = @WG "(dx,dy,dz,x,y,z) -> screened_coulomb_weightfunc(dx,dy,dz,3.9,5.0,1.0)" NN = (3,3,3)
wg_anisotropic = @WG "(dx,dy,dz,x,y,z) -> anisotropic_layered_weightfunc(dx,dy,dz,3.9,25.0,2.0,1.0)" NN = (3,3,3)
wg_strain = @WG "(dx,dy,dz,x,y,z) -> strain_modulated_weightfunc(dx,dy,dz,x,y,z,3.9,0.01,1.0)" NN = (2,2,2)

println("Advanced weight functions created!")
println("Usage: genAdj!(g[1], wg_exponential)  # or other wg_xxx")
println("Note: Physical parameters of these functions need to be adjusted for specific materials!")

In [ ]:
x_n = random_partition(xL, 20)
y_n = random_partition(yL, 20)
z_n = random_partition(zL, 3)
coef_x =randAround(1, 0.2, 20)
coef_y =randAround(1, 0.2, 20)
coef_z =randAround(1, 0.2, 3)
println(x_n)
println(y_n)
println(z_n)
println(coef_x)
println(coef_y)
println(coef_z)


function weightfunc_glass_anti(dx, dy, dz, x, y, z, x_n, y_n,z_n, coef_x,coef_y, coef_z)
    prefac = 1
    ix = findfirst(v -> x ≤ v, x_n)
    iy = findfirst(v -> y ≤ v, y_n)
    iz = findfirst(v -> z ≤ v, z_n)
    if isnothing(ix) || isnothing(iy)
        error("out of range")
    end
    dr2 = (coef_x[ix] * dx)^2 + (coef_y[iy] * dy)^2 +(coef_z[iz]* dz)^2
    if (abs(dx) + abs(dy)) % 2 != 0  
        prefac = -1
    else  
        prefac = 1
    end
    return prefac / dr2
end

function weightfunc_glass_ferro(dx, dy, dz, x, y, z, x_n, y_n,z_n, coef_x,coef_y, coef_z)
    prefac = 1
    ix = findfirst(v -> x ≤ v, x_n)
    iy = findfirst(v -> y ≤ v, y_n)
    iz = findfirst(v -> z ≤ v, z_n)
    if isnothing(ix) || isnothing(iy)
        error("out of range")
    end
    dr2 = (coef_x[ix] * dx)^2 + (coef_y[iy] * dy)^2 +(coef_z[iz]* dz)^2
    prefac = 1
    return prefac / dr2
end

function weightfunc_AntiWithFerro(dx, dy, dz, x, y, z, x_n, y_n,z_n, coef_x, coef_y, coef_z)
    prefac = 1
    ix = findfirst(v -> x ≤ v, x_n)
    iy = findfirst(v -> y ≤ v, y_n)
    iz = findfirst(v -> z ≤ v, z_n)
    if isnothing(ix) || isnothing(iy)
        error("out of range")
    end
    dr2 = (coef_x[ix] * dx)^2 + (coef_y[iy] * dy)^2 +(coef_z[iz]* dz)^2
    if ix % 2 == 0 && iy %2==0
        prefac=1
    elseif xor(isodd(ix), isodd(iy))
        if (abs(dx) + abs(dy)) % 2 != 0  
            prefac = -1
        else  
            prefac = 1
        end
    end
    return prefac / dr2
end

wg_glass1 = @WG "(dx,dy,dz,x,y,z) -> weightfunc_glass_ferro(dx,dy,dz,x,y,z,x_n, y_n, z_n, coef_x,coef_y,coef_z)" NN = (2,2,2)
wg_glass2 = @WG "(dx,dy,dz,x,y,z) -> weightfunc_glass_anti(dx,dy,dz,x,y,z, x_n, y_n, z_n, coef_x,coef_y,coef_z)" NN = (2,2,2)
wg_glass3 = @WG "(dx,dy,dz,x,y,z) -> weightfunc_AntiWithFerro(dx, dy, dz, x, y, z, x_n, y_n, z_n, coef_x, coef_y, coef_z)" NN = (2,2,2)


**Simulation Examples**

Now we'll run three different simulations to demonstrate how different weight functions affect the hysteresis behavior:

1. **Example 1**: Standard ferromagnetic coupling with uniform interactions
2. **Example 2**: Spin glass with antiferromagnetic checkerboard pattern  
3. **Example 3**: Mixed ferro/antiferromagnetic domains

Each simulation uses the same external field protocol (triangular pulse) but different internal coupling patterns. Compare the resulting hysteresis loops to see how the microscopic interactions influence macroscopic behavior.

In [19]:
# Example 1: normal weight function (ferro)
g = IsingGraph(xL, yL, zL, type = Continuous(), periodic = (:x,:y))

# Set visual marker size for clarity
II.makie_markersize[] = 0.3

# Launch interactive visualization (will remain idle until we create a process)
interface(g)

# Set depolarization (DP) field:
# Smaller c => stronger depolarization penalty (suppresses net polarization, can pinch loops)
# Larger  c => weaker depolarization (permits larger remanent polarization)
# Here: c=60000 (weak DP) for a clear baseline hysteresis loop.
g.hamiltonian = Ising(g) + DepolField(g, c=60000, left_layers=1, right_layers=1)

# Make external field parameter :b homogeneous & mutable so pulse routines can update hamiltonian.b[]
g.hamiltonian = sethomogenousparam(g.hamiltonian, :b)

# Apply a moderate negative self-energy bias.
# This favors polarized states (ferroelectric tendency),
# making flips toward zero polarization less likely
# and stabilizing domain alignment.

homogeneousself!(g, -1)

### We set Temperature to 1.5
settemp(g,1.5)

### Change nearest neighbours
####################################################################################
wg1 = @WG "(dx,dy,dz,x,y,z) -> weightfunc1(dx,dy,dz,1,1,1)" NN = (2,2,2)
####################################################################################
genAdj!(g[1], wg1)


fullsweep = xL*yL*zL
Time_fctr = 0.2
SpeedRate = Int(Time_fctr*fullsweep)
risepoint=500
Amptitude =20
# risepoint = round(Int, Amptitude/0.01)

#### Run with TrianlePulseA
PulseN = 2
Pulsetime = (PulseN * 4 + 2) * risepoint * SpeedRate
compalgo = CompositeAlgorithm((Metropolis, TrianglePulseA), (1, SpeedRate))
createProcess(g, compalgo, lifetime =Pulsetime, amp = Amptitude, numpulses = PulseN, rise_point=risepoint)
### estimate time
est_remaining(process(g))

# Wait until it is done
args = process(g) |> fetch # If you want to close ctr+c
# args = process(g) |> getargs
# EnergyG= args.all_Es;
voltage= args.x
Pr= args.y;

# w1=newmakie(lines, Pr, EnergyG)
w2=newmakie(lines, voltage, Pr)
# w3=newmakie(lines,Pr)


Graph: Continuous() IsingLayer 1 with size (100, 100, 10) and stateset (-1.0f0, 1.0f0)

 with connections:
 and 0 defects
InteractiveIsing.LayoutPanel
Base.RefValue{Float32}(0.0f0)
WG: WeightGenerator with
	 NN: 				(2, 2, 2)
	 func: 				(dx,dy,dz,x,y,z) -> weightfunc1(dx,dy,dz,1,1,1)
D: 3
Estimated time to completion: 0:1:18
Of which remaining: 0:1:18


**Results Analysis - Example 1 vs Example 2**

Compare the hysteresis loops from the two simulations:

- **Example 1 (ferro)**: Should show clean, symmetric loops with sharp switching
- **Example 2 (spin glass anti)**: Expect more rounded loops, possible pinching, and asymmetric behavior

The spin glass system creates **competing interactions** that lead to:
- Frustrated energy landscapes
- Multiple metastable states  
- History-dependent switching behavior
- Reduced coercive field uniformity

In [ ]:
# Example 2：spin glass anti alignment
g_glass_anti = IsingGraph(xL, yL, zL, type = Continuous(), periodic = (:x,:y))

# Set visual marker size for clarity
II.makie_markersize[] = 0.3

# Launch interactive visualization (will remain idle until we create a process)
interface(g_glass_anti)

# Set depolarization (DP) field:
# Smaller c => stronger depolarization penalty (suppresses net polarization, can pinch loops)
# Larger  c => weaker depolarization (permits larger remanent polarization)
# Here: c=60000 (weak DP) for a clear baseline hysteresis loop.
g_glass_anti.hamiltonian = Ising(g_glass_anti) + DepolField(g_glass_anti, c=60000, left_layers=1, right_layers=1)

# Make external field parameter :b homogeneous & mutable so pulse routines can update hamiltonian.b[]
g_glass_anti.hamiltonian = sethomogenousparam(g_glass_anti.hamiltonian, :b)

# Apply a moderate negative self-energy bias.
# This favors polarized states (ferroelectric tendency),
# making flips toward zero polarization less likely
# and stabilizing domain alignment.
homogeneousself!(g_glass_anti, -1)

### We set Temperature to 1.5
settemp(g_glass_anti,1.5)


genAdj!(g_glass_anti[1], wg_glass2)

fullsweep = xL*yL*zL
Time_fctr = 0.2
SpeedRate = Int(Time_fctr*fullsweep)
risepoint=500
Amptitude =20
# risepoint = round(Int, Amptitude/0.01)

#### Run with TrianlePulseA
PulseN = 2
Pulsetime = (PulseN * 4 + 2) * risepoint * SpeedRate
compalgo = CompositeAlgorithm((Metropolis, TrianglePulseA), (1, SpeedRate))
createProcess(g_glass_anti, compalgo, lifetime =Pulsetime, amp = Amptitude, numpulses = PulseN, rise_point=risepoint)

### estimate time
est_remaining(process(g_glass_anti))

# Wait until it is done
args = process(g_glass_anti) |> fetch # If you want to close ctr+c
# args = process(g_glass_anti) |> getargs
# EnergyG= args.all_Es;
voltage= args.x
Pr= args.y;

# w1=newmakie(lines, Pr, EnergyG)
w_anti_2=newmakie(lines, voltage, Pr)
# w_anti_3=newmakie(lines,Pr)
inlineplot() do 
    lines(voltage, Pr)
end

**Example 3: Mixed Ferro/Antiferromagnetic Domains**

This simulation demonstrates the most complex case where:
- **Even-indexed domains**: Pure ferromagnetic behavior
- **Odd-indexed domains**: Mixed ferro/antiferromagnetic patterns based on XOR logic
- **Domain boundaries**: Create additional frustration and pinning sites

Expected behavior:
- Intermediate loop characteristics between Examples 1 and 2
- Possible multi-step switching due to domain heterogeneity
- Enhanced coercive field distribution

In [ ]:
# Example 3：spin glass with mixed interactions
g_GlassMix = IsingGraph(xL, yL, zL, type = Continuous(), periodic = (:x,:y))

# Set visual marker size for clarity
II.makie_markersize[] = 0.3

# Launch interactive visualization (will remain idle until we create a process)
interface(g_GlassMix)

# Set depolarization (DP) field:
# Smaller c => stronger depolarization penalty (suppresses net polarization, can pinch loops)
# Larger  c => weaker depolarization (permits larger remanent polarization)
# Here: c=60000 (weak DP) for a clear baseline hysteresis loop.
g_GlassMix.hamiltonian = Ising(g_GlassMix) + DepolField(g_GlassMix, c=60000, left_layers=1, right_layers=1)

# Make external field parameter :b homogeneous & mutable so pulse routines can update hamiltonian.b[]
g_GlassMix.hamiltonian = sethomogenousparam(g_GlassMix.hamiltonian, :b)

# Apply a moderate negative self-energy bias.
# This favors polarized states (ferroelectric tendency),
# making flips toward zero polarization less likely
# and stabilizing domain alignment.
homogeneousself!(g_GlassMix, -1)

### We set Temperature to 1.5
settemp(g_GlassMix,1.5)


genAdj!(g_GlassMix[1], wg_glass3)

fullsweep = xL*yL*zL
Time_fctr = 0.2
SpeedRate = Int(Time_fctr*fullsweep)
risepoint=500
Amptitude =20
# risepoint = round(Int, Amptitude/0.01)

#### Run with TrianlePulseA
PulseN = 2
Pulsetime = (PulseN * 4 + 2) * risepoint * SpeedRate
compalgo = CompositeAlgorithm((Metropolis, TrianglePulseA), (1, SpeedRate))
createProcess(g_GlassMix, compalgo, lifetime =Pulsetime, amp = Amptitude, numpulses = PulseN, rise_point=risepoint)

### estimate time
est_remaining(process(g_GlassMix))

# Wait until it is done
args = process(g_GlassMix) |> fetch # If you want to close ctr+c
# args = process(g_GlassMix) |> getargs
# EnergyG= args.all_Es;
voltage= args.x
Pr= args.y;

w_Mix_2=newmakie(lines, voltage, Pr)    
# w_Mix_3=newmakie(lines,Pr)
inlineplot() do 
    lines(voltage, Pr)
end